# Importing libraries

In [1]:
import numpy as np
import pandas as pd
from pandas.io.json import json_normalize
import json
from geopy.geocoders import Nominatim
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
import folium 

# reading the data prepared in "clustering_no2"
It's the previous task

In [54]:
neighborhoods=pd.read_csv('data.csv', index_col=0)
neighborhoods.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494


# creating Toronto's map to see general picture with our points from dataframe
from here, i will use a lot of code from our NY lab \
As soon as all of us have passed it, it will help to understand what is going on

In [11]:
# create map using latitude and longitude values
latitude=43.72
longitude=-79.32
map_trnt = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map (this code was copypasted, but dont blame me, i know how it works xD )
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['PostalCode'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_trnt)  
    
map_trnt #uncomment to see the map

# getting venues for every borough
for the sake of security I will simply upload the file with results from the disk \
I filled this file on my own, using API just like in NY lab, but I'm not sure that it is a good idea to share my forsquare credentials here in this work. \
Thus all the code with requests to forsquare is not presented here.

In [19]:
toronto_venues=pd.read_csv('toronto_venues.csv', index_col=0)
print(toronto_venues.shape)
toronto_venues.head()

(2241, 7)


,PostalCode,PostalCode Latitude,PostalCode Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,M3A,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,M3A,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,M4A,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,M4A,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,M4A,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


In [20]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
#and that is less than in Manhattan

There are 272 uniques categories.


# Making data pretty
In order to clarify something before implementing clustering algorythms we need to transform our data in the way they would be able to tell the story for us.

Also it's the same format we used in NY lab, thus we are able to use same scrypts

In [22]:
# one hot encoding
toronto_ohe = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_ohe['PostalCode'] = toronto_venues['PostalCode'] 

# move neighborhood column to the first column
fixed_columns = [toronto_ohe.columns[-1]] + list(toronto_ohe.columns[:-1])
toronto_ohe = toronto_ohe[fixed_columns]

#now let's see the result
print(toronto_ohe.shape)
toronto_ohe.head()

(2241, 273)


,PostalCode,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,M3A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,M4A,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [31]:
toronto_grouped = toronto_ohe.groupby('PostalCode').mean().reset_index()
toronto_grouped
# that is what we are going to use for clustering

,PostalCode,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wings Joint,Women's Store,Yoga Studio
0,M1B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
1,M1C,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
2,M1E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
3,M1G,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
4,M1H,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,M9N,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
97,M9P,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
98,M9R,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.0,0.0,0.0,0.0,0.0,0.0
99,M9V,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.125,0.0,0.0,0.0,0.0,0.0,0.0


Now we can learn something by looking through most popular places for every postal code

In [25]:
num_top_venues = 10

for code in toronto_grouped['PostalCode']:
    print("----"+code+"----")
    temp = toronto_grouped[toronto_grouped['PostalCode'] == code].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----M1B----
                        venue  freq
0        Fast Food Restaurant   0.5
1                  Print Shop   0.5
2               Metro Station   0.0
3  Modern European Restaurant   0.0
4           Mobile Phone Shop   0.0
5          Miscellaneous Shop   0.0
6   Middle Eastern Restaurant   0.0
7          Mexican Restaurant   0.0
8                 Men's Store   0.0
9         Monument / Landmark   0.0


----M1C----
                        venue  freq
0               Moving Target  0.33
1  Construction & Landscaping  0.33
2                         Bar  0.33
3           Accessories Store  0.00
4          Mexican Restaurant  0.00
5  Modern European Restaurant  0.00
6           Mobile Phone Shop  0.00
7          Miscellaneous Shop  0.00
8   Middle Eastern Restaurant  0.00
9               Metro Station  0.00


----M1E----
                 venue  freq
0    Electronics Store  0.22
1   Mexican Restaurant  0.11
2                  Spa  0.11
3          Pizza Place  0.11
4       Breakfast Spot 

9  Modern European Restaurant  0.00


----M4R----
                  venue  freq
0        Clothing Store  0.14
1   Sporting Goods Shop  0.09
2           Coffee Shop  0.09
3           Yoga Studio  0.05
4  Gym / Fitness Center  0.05
5    Mexican Restaurant  0.05
6                 Diner  0.05
7          Dessert Shop  0.05
8        Cosmetics Shop  0.05
9                  Park  0.05


----M4S----
                 venue  freq
0         Dessert Shop  0.09
1       Sandwich Place  0.09
2          Pizza Place  0.09
3   Italian Restaurant  0.06
4          Coffee Shop  0.06
5     Sushi Restaurant  0.06
6                  Gym  0.06
7                 Café  0.06
8  Fried Chicken Joint  0.03
9      Thai Restaurant  0.03


----M4T----
                             venue  freq
0                       Restaurant  0.33
1                       Playground  0.33
2                     Intersection  0.33
3               Mexican Restaurant  0.00
4  Molecular Gastronomy Restaurant  0.00
5       Modern European Res

                  venue  freq
0                   Gym  0.08
1                Bakery  0.08
2   Fried Chicken Joint  0.08
3           Flower Shop  0.08
4          Liquor Store  0.08
5  Fast Food Restaurant  0.08
6              Pharmacy  0.08
7           Coffee Shop  0.08
8            Restaurant  0.08
9                  Café  0.08


----M8W----
                  venue  freq
0           Pizza Place  0.22
1                   Gym  0.11
2          Skating Rink  0.11
3          Dance Studio  0.11
4           Coffee Shop  0.11
5                   Pub  0.11
6        Sandwich Place  0.11
7              Pharmacy  0.11
8      Video Game Store  0.00
9  Other Great Outdoors  0.00


----M8X----
                             venue  freq
0                             Pool   0.5
1                            River   0.5
2                    Metro Station   0.0
3  Molecular Gastronomy Restaurant   0.0
4       Modern European Restaurant   0.0
5                Mobile Phone Shop   0.0
6               Miscellan

In [27]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [83]:
# create columns according to number of top venues
columns = ['PostalCode']
for ind in np.arange(num_top_venues):
        columns.append('#{} Most Common Venue'.format(ind+1))

# create a new dataframe
venues_sorted = pd.DataFrame(columns=columns)
venues_sorted['PostalCode'] = toronto_grouped['PostalCode']

for ind in np.arange(toronto_grouped.shape[0]):
    venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

v_4=venues_sorted# creating a copy for the future
venues_sorted.head()

,PostalCode,#1 Most Common Venue,#2 Most Common Venue,#3 Most Common Venue,#4 Most Common Venue,#5 Most Common Venue,#6 Most Common Venue,#7 Most Common Venue,#8 Most Common Venue,#9 Most Common Venue,#10 Most Common Venue
0,M1B,Fast Food Restaurant,Print Shop,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
1,M1C,Moving Target,Construction & Landscaping,Bar,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
2,M1E,Electronics Store,Medical Center,Mexican Restaurant,Spa,Breakfast Spot,Pizza Place,Rental Car Location,Intersection,Dim Sum Restaurant,Diner
3,M1G,Coffee Shop,Pharmacy,Korean Restaurant,Yoga Studio,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
4,M1H,Gas Station,Bakery,Fried Chicken Joint,Caribbean Restaurant,Athletics & Sports,Thai Restaurant,Bank,Hakka Restaurant,Dumpling Restaurant,Drugstore


As we can see from here, codes that are close to each other in string way, are close in common venues. \
And as soon as we know that they are close to each other in the real world, it makes sense

# And now we do the clustering

In [72]:
# set number of clusters
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_[0:10])
print(set(kmeans.labels_))

[0 0 0 0 0 1 0 0 0 0]
{0, 1, 2, 3, 4}


In [73]:
# add clustering labels
venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(venues_sorted.set_index('PostalCode'), on='PostalCode')

toronto_merged.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,#1 Most Common Venue,#2 Most Common Venue,#3 Most Common Venue,#4 Most Common Venue,#5 Most Common Venue,#6 Most Common Venue,#7 Most Common Venue,#8 Most Common Venue,#9 Most Common Venue,#10 Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2.0,Park,Food & Drink Shop,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Coffee Shop,Hockey Arena,French Restaurant,Portuguese Restaurant,Yoga Studio,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0.0,Coffee Shop,Park,Bakery,Pub,Restaurant,Breakfast Spot,Café,Mexican Restaurant,Theater,Dessert Shop
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,0.0,Furniture / Home Store,Clothing Store,Accessories Store,Coffee Shop,Miscellaneous Shop,Carpet Store,Fraternity House,Vietnamese Restaurant,Boutique,Dumpling Restaurant
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,0.0,Coffee Shop,Diner,Park,Gym,Sushi Restaurant,Yoga Studio,Beer Bar,Smoothie Shop,Seafood Restaurant,Sandwich Place


In [74]:
toronto_merged.dropna(inplace=True)
toronto_merged['Cluster Labels']=toronto_merged['Cluster Labels'].astype('int32')
set(toronto_merged['Cluster Labels'])

{0, 1, 2, 3, 4}

# Now let's see what we have got here

In [62]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['PostalCode'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

# Outliers
It's clearly seen, that we have one common type, two more massive ones, and two outliers
Let's look for the features what makes them look different (and if there is none, we'd change the number of clusters)

In [63]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,#1 Most Common Venue,#2 Most Common Venue,#3 Most Common Venue,#4 Most Common Venue,#5 Most Common Venue,#6 Most Common Venue,#7 Most Common Venue,#8 Most Common Venue,#9 Most Common Venue,#10 Most Common Venue
1,North York,0,Coffee Shop,Hockey Arena,French Restaurant,Portuguese Restaurant,Yoga Studio,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
2,Downtown Toronto,0,Coffee Shop,Park,Bakery,Pub,Restaurant,Breakfast Spot,Café,Mexican Restaurant,Theater,Dessert Shop
3,North York,0,Furniture / Home Store,Clothing Store,Accessories Store,Coffee Shop,Miscellaneous Shop,Carpet Store,Fraternity House,Vietnamese Restaurant,Boutique,Dumpling Restaurant
4,Queen's Park,0,Coffee Shop,Diner,Park,Gym,Sushi Restaurant,Yoga Studio,Beer Bar,Smoothie Shop,Seafood Restaurant,Sandwich Place
6,Scarborough,0,Fast Food Restaurant,Print Shop,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
...,...,...,...,...,...,...,...,...,...,...,...,...
98,Etobicoke,0,River,Pool,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Yoga Studio,Deli / Bodega
99,Downtown Toronto,0,Sushi Restaurant,Coffee Shop,Gay Bar,Japanese Restaurant,Restaurant,Café,Hotel,Mediterranean Restaurant,Gym,Gastropub
100,East Toronto,0,Smoke Shop,Pizza Place,Skate Park,Brewery,Farmers Market,Spa,Fast Food Restaurant,Burrito Place,Restaurant,Recording Studio
101,Etobicoke,0,Pool,Business Service,Baseball Field,Department Store,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Yoga Studio


In [64]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,#1 Most Common Venue,#2 Most Common Venue,#3 Most Common Venue,#4 Most Common Venue,#5 Most Common Venue,#6 Most Common Venue,#7 Most Common Venue,#8 Most Common Venue,#9 Most Common Venue,#10 Most Common Venue
32,Scarborough,1,Playground,Yoga Studio,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore


As we can see, this one is different. There is no other borough that has similar venue list. Thus it is okay.

In [65]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,#1 Most Common Venue,#2 Most Common Venue,#3 Most Common Venue,#4 Most Common Venue,#5 Most Common Venue,#6 Most Common Venue,#7 Most Common Venue,#8 Most Common Venue,#9 Most Common Venue,#10 Most Common Venue
0,North York,2,Park,Food & Drink Shop,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
21,York,2,Park,Fast Food Restaurant,Market,Women's Store,Gift Shop,German Restaurant,Electronics Store,Eastern European Restaurant,Golf Course,Dumpling Restaurant
35,East York,2,Park,Coffee Shop,Convenience Store,Ethiopian Restaurant,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Deli / Bodega
40,North York,2,Park,Airport,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
49,North York,2,Park,Bakery,Construction & Landscaping,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore
52,North York,2,Park,Piano Bar,Doner Restaurant,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
64,York,2,Convenience Store,Park,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Yoga Studio
66,North York,2,Park,Bank,Convenience Store,Empanada Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Drugstore,Department Store,Donut Shop
68,Central Toronto,2,Park,Jewelry Store,Sushi Restaurant,Trail,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store
85,Scarborough,2,Park,Arts & Crafts Store,Playground,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop


In [66]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,#1 Most Common Venue,#2 Most Common Venue,#3 Most Common Venue,#4 Most Common Venue,#5 Most Common Venue,#6 Most Common Venue,#7 Most Common Venue,#8 Most Common Venue,#9 Most Common Venue,#10 Most Common Venue
45,North York,3,Cafeteria,Donut Shop,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore,College Rec Center


But this one is clearly an outlier, because I see no difference between this one and the cluster 0. Thus, we should run the model one more time, but with the number of clusters equals to 4

In [67]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 4, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,#1 Most Common Venue,#2 Most Common Venue,#3 Most Common Venue,#4 Most Common Venue,#5 Most Common Venue,#6 Most Common Venue,#7 Most Common Venue,#8 Most Common Venue,#9 Most Common Venue,#10 Most Common Venue
46,North York,4,Grocery Store,Shopping Mall,Bank,Hotel,Park,Electronics Store,Empanada Restaurant,Eastern European Restaurant,Dumpling Restaurant,Drugstore
60,North York,4,Grocery Store,Liquor Store,Gym / Fitness Center,Athletics & Sports,Donut Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Doner Restaurant
61,Central Toronto,4,Park,Gym / Fitness Center,Swim School,Bus Line,Yoga Studio,Doner Restaurant,Dim Sum Restaurant,Diner,Discount Store,Dog Run
63,York,4,Grocery Store,Caribbean Restaurant,Bus Line,Brewery,Donut Shop,Diner,Discount Store,Dog Run,Doner Restaurant,Drugstore


# Give me baby one more time

In [82]:
# set number of clusters (now less than the last time)
kclusters = 4

toronto_grouped_clustering_4 = toronto_grouped.drop('PostalCode', 1)

# run k-means clustering
kmeans_4 = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering_4)

# check cluster labels generated for each row in the dataframe
print(kmeans_4.labels_[0:10])
print(set(kmeans_4.labels_))

[0 0 0 0 0 1 0 0 0 0]
{0, 1, 2, 3}


In [84]:
# add clustering labels

v_4.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged_4 = neighborhoods

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged_4 = toronto_merged_4.join(v_4.set_index('PostalCode'), on='PostalCode')

toronto_merged_4.head() # check the last columns!

,PostalCode,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,#1 Most Common Venue,#2 Most Common Venue,#3 Most Common Venue,#4 Most Common Venue,#5 Most Common Venue,#6 Most Common Venue,#7 Most Common Venue,#8 Most Common Venue,#9 Most Common Venue,#10 Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2.0,Park,Food & Drink Shop,Yoga Studio,Doner Restaurant,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dog Run,Donut Shop
1,M4A,North York,Victoria Village,43.725882,-79.315572,0.0,Coffee Shop,Hockey Arena,French Restaurant,Portuguese Restaurant,Yoga Studio,Dog Run,Department Store,Dessert Shop,Dim Sum Restaurant,Diner
2,M5A,Downtown Toronto,Harbourfront,43.654260,-79.360636,0.0,Coffee Shop,Park,Bakery,Pub,Restaurant,Breakfast Spot,Café,Mexican Restaurant,Theater,Dessert Shop
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763,0.0,Furniture / Home Store,Clothing Store,Accessories Store,Coffee Shop,Miscellaneous Shop,Carpet Store,Fraternity House,Vietnamese Restaurant,Boutique,Dumpling Restaurant
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494,0.0,Coffee Shop,Diner,Park,Gym,Sushi Restaurant,Yoga Studio,Beer Bar,Smoothie Shop,Seafood Restaurant,Sandwich Place


sorry, but i was too tired to create another copy of v4, that's why u had to redo the paragraph just before "and now we do the clustering" BEFORE u run the paragraph above

In [85]:
toronto_merged_4.dropna(inplace=True)
toronto_merged_4['Cluster Labels']=toronto_merged_4['Cluster Labels'].astype('int32')
set(toronto_merged_4['Cluster Labels'])

{0, 1, 2, 3, 4}

In [86]:
# create map
map_clusters_4 = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged_4['Latitude'], toronto_merged_4['Longitude'], toronto_merged_4['PostalCode'], toronto_merged_4['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters_4)
       
map_clusters_4

# aaaaand we also have two outliers
And what is interesting, they are tha same \
cool, ehh